In [27]:
import functools

# Generalizing code into a decorator

See also `decwrapornot.ipynb` for more general information about when decorators should define and return a wrapper function vs. modifying or using information from the function passed as its argument and passing that through.

## Example 1: `@peek_arg`

We begin with this function that squares its argument:

In [28]:
def square(number):
    """Find the square of a value."""
    return number**2

In [29]:
square(3)

9

Suppose we want&mdash;temporarily or permanently&mdash;to instrument it so it prints its argument when it is called:

In [30]:
def square(number):
    """Find the square of a value. Log the call."""
    print(f'square({number})')
    return number**2

In [31]:
square(3)

square(3)


9

If this is all we want to do&mdash;add instrumentation to a single function&mdash;then a decorator may be overkill. Just adding a line of code to `square` is sufficient. But if we want to make this change to multiple functions, or if the change were more involved and we wanted to separate its logic from that of the function it instruments, then it is reasonable to consider extracting it to a decorator.

We want to be able to put a decoration, `@peek_arg`, at the top of the definition of `square`, to cause `square` to have the added behavior of printing out how it was called. It would often be best to make `peek_arg` work on any function of any arity (and with or without keyword arguments), but for simplicity let&rsquo;s assume we are only interested in using it to instrument functions that we call with a single positional argument and no keyword arguments.

Besides that, though, we don&rsquo;t know anything about the behavior of functions that we will decorate with `@peek_arg`. More precisely, whether or not we know that, we don&rsquo;t want to `@peek_arg` to rely on those details, since the whole point is to separate the logging functionality it performs from the behavior of the functions it will decorate.

Because the behavior of the function `peek_arg` returns adds to the behavior of the function passed to it:

- `peek_arg` will return a different function than the one passed to it, because the behavior of the function it returns will differ from the behavior of the function passed to it.

- The behavior of the function `peek_arg` will return *includes* the behavior of the function passed to it. So the function `peek_arg` returns will call the function passed to `peek_arg`.

To begin extracting the printing code to a decorator, we can make a function, `wrapper`, that works just like that:

In [32]:
def square(number):
    """Find the square of a value."""
    return number**2

In [33]:
def wrapper(number):
    """Find the square of a value. But first, report the function call."""
    print(f'square({number})')
    return square(number)

In [34]:
wrapper(3)

square(3)


9

Right now, `wrapper` depends on `square` beyond just referring to it by name, because it hard-codes the substring `square` in the interpolated string (the f-string) it prints. We can fix this by using `square.__name__` instead:

In [35]:
def wrapper(number):
    """Find the square of a value. But first, report the function call."""
    print(f'{square.__name__}({number})')
    return square(number)

In [36]:
wrapper(3)

square(3)


9

Now that `wrapper`&rsquo;s only dependence on `square` is by the variable name `square`, we can create a factory function that takes this as a parameter and makes functions like `wrapper`. Doing this gives us:

In [37]:
def peek_arg(square):
    """Decorator wrapping a unary function and showing calls to it."""
    def wrapper(number):
        print(f'{square.__name__}({number})')
        return square(number)
    
    return wrapper

This no longer is specific to squaring, nor to functions whose argument is a number, so we can pick more general parameter/names:

In [38]:
def peek_arg(func):
    """Decorator wrapping a unary function and showing calls to it."""
    def wrapper(arg):
        print(f'{func.__name__}({arg})')
        return func(arg)
    
    return wrapper

We should also decorate `peek_arg` with `@functools.wraps(func)` so values of some important metadata attributes are copied from `func` to `wrapper`. This is desirable because `wrapper` will replace `func`&mdash;for example, it will be the function actually assigned to `square` when we decorate the definition of `square` with `@peek_arg`. Without doing this, anything that uses those metadata, such as doctest runners and the `help` builtin, will not work.

For more information on `functools.wraps`, see [its documentation](https://docs.python.org/3/library/functools.html#functools.wraps) and the **&ldquo;Example 3: <code>@give_metadata_from(*wrapped*)</code>&rdquo;** section below.

In [39]:
def peek_arg(func):
    """Decorator wrapping a unary function and showing calls to it."""
    @functools.wraps(func)
    def wrapper(arg):
        print(f'{func.__name__}({arg})')
        return func(arg)
    
    return wrapper

It works on `square`:

In [40]:
@peek_arg
def square(number):
    """Find the square of a value. Log the call."""
    return number**2

In [41]:
square(3)

square(3)


9

But just as well on other functions that are conceptually unrelated to `square`:

In [42]:
@peek_arg
def say_hi(name):
    """Say hi to someone, given their name. Log the call."""
    print(f'Hi, {name}!')

In [43]:
say_hi('Bob')

say_hi(Bob)
Hi, Bob!


Because we used `functools.wraps`, the metadata are available:

In [44]:
help(square)

Help on function square in module __main__:

square(number)
    Find the square of a value. Log the call.



In [45]:
help(say_hi)

Help on function say_hi in module __main__:

say_hi(name)
    Say hi to someone, given their name. Log the call.



## Example 2: `@call`

Sometimes we define a function just to have a local scope, and immediately call it. Other times, we will use the function later, but we still want to call it immediately. Both scenarios are most realistic in the setting of Jupyter notebooks.

We begin with this function that demonstrates how `_` is not assigned to when used as a `match`-`case` discard but is assigned to when used in tuple unpacking assignment:

In [46]:
def demonstrate_underscore():
    """Show the behavior of _ in various contexts."""
    match [10, 20]:
        case _, second:
            pass
    print(locals())

    _, second = [10, 20]
    print(locals())

demonstrate_underscore()

{'second': 20}
{'second': 20, '_': 10}


Already we can see that we don&rsquo;t need a wrapper and shouldn&rsquo;t use one: the functionality we want to augment the definition of `demonstrate_underscore` with is *already separate from that definition*. We&rsquo;re not trying to change what happens when `demonstrate_underscore` is called&mdash;instead, we want to add behavior to *the act of defining it*.

We want to be able to decorate the definition with `@call` to produce the effect of the above code: defining the function, and calling it. Because we are not changing `demonstrate_underscore`&rsquo;s behavior, we won&rsquo;t return a different function, but instead just the same function.

Our `@call` decorator starts out as just an identity function&mdash;one that takes an argument, in this case a function, and returns it:

In [47]:
def call(demonstrate_underscore):
    # FIXME: This needs to do something.
    return demonstrate_underscore

In [48]:
@call
def demonstrate_underscore():
    """Show the behavior of _ in various contexts."""
    match [10, 20]:
        case _, second:
            pass
    print(locals())

    _, second = [10, 20]
    print(locals())

demonstrate_underscore()

{'second': 20}
{'second': 20, '_': 10}


Now we extract the call to `demonstrate_underscore` into the decorator:

In [49]:
def call(demonstrate_underscore):
    demonstrate_underscore()
    return demonstrate_underscore

And change the name of its parameter, since it is in no way specific to `demonstrate_underscore`, but instead works with any function that can be called with no arguments:

In [50]:
def call(func):
    func()
    return func

Now we can decorate the definition of `demonstrate_underscore` with `@call` instead of calling it after defining it:

In [51]:
@call
def demonstrate_underscore():
    """Show the behavior of _ in various contexts."""
    match [10, 20]:
        case _, second:
            pass
    print(locals())

    _, second = [10, 20]
    print(locals())

{'second': 20}
{'second': 20, '_': 10}


But `@call` also works with other unary functions that are not conceptually related to `demonstrate_underscore`:

In [52]:
@call
def hello():
    print('Hello, world!')

Hello, world!
